# Ethereum transaction OpenSSL signing

Create, sign and send ethereum transaction by the hard way

**ToDo:** use own dir for temp files or make all subprocess operations without generatin files in project folder

## 1 Imports, Web3 connection
Now connect to Ethereum provider via web3 RPC

In [1]:
import subprocess
from web3 import Web3

from eth_account._utils.transactions import (
    encode_transaction,
    serializable_unsigned_transaction_from_dict,
)

from eth_keys import (
    keys,
)

from eth_keys.datatypes import (  
    PublicKey,
)

from eth_keys.backends.native.ecdsa import (
    ecdsa_raw_verify,
    ecdsa_raw_recover,
    decode_public_key,
    private_key_to_public_key,
)

In [2]:
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
CHAIN_ID = 1

## 2.1 Generate private key via OpenSSL (optional)

In [ ]:
subprocess.run('openssl ecparam -name secp256k1 -genkey -noout -out private_key.pem', shell=True).returncode
with open('private_key.pem') as f:
    for line in f:
        print(line, end='')

In [ ]:
raw_output = subprocess.run('openssl ec -in private_key.pem -text -noout', shell=True, capture_output=True).stdout.decode()
print(raw_output)

In [ ]:
private_key_hexstr = ''.join([x.replace(' ','').replace(':','') for x in raw_output.split('\n')[2:5]])
print('priv: ' + private_key_hexstr)
public_key_hexstr = ''.join([x.replace(' ','').replace(':','') for x in raw_output.split('\n')[6:11]])
print('pub: ' + public_key_hexstr)

## 2.2 Import raw private key (e.g. from ganache) and create locally its .pem file
It should be in hex format without `0x` prefix

In [3]:
# this adress is 0 account from ganache
private_key_hexstr = '44d01ff7ab5e778dac355876d5200b1f0d4f8d9bbd0f15723e7683f9c6ee8a78'

public_key_hexstr = '04' + Web3.toHex(private_key_to_public_key(Web3.toBytes(hexstr=private_key_hexstr)))[2:]
subprocess.run(f'echo 30740201010420 {private_key_hexstr} a00706052b8104000aa144034200 {public_key_hexstr} | xxd -r -p > raw_pem_key', shell=True).returncode
subprocess.run('openssl ec -inform d < raw_pem_key > private_key.pem', shell=True).returncode

0

## 3 Create ethereum transaction dict

In [4]:
w3.eth.defaultAccount = w3.eth.accounts[0]

nonce = w3.eth.getTransactionCount(w3.eth.defaultAccount)
transaction_dict = {
    #'from': w3.eth.accounts[0],
    'to': Web3.toChecksumAddress(w3.eth.accounts[1]),
    'value': 1,
    'gas': 200000,
    'gasPrice': 0,
    'nonce': nonce,
    'chainId': CHAIN_ID
}

nonce

4

## 4.1 Sign transaction dict via web3.py (optional)

In [ ]:
signed_tx = w3.eth.account.signTransaction(transaction_dict, private_key_hexstr)
print('signed tx hex:\n' + w3.toHex(signed_tx.rawTransaction) + '\n')
print('signed tx hash hex:\n' + w3.toHex(signed_tx.hash) + '\n')
print('tx ecdsa signature, (v, r, s):')
print(w3.toHex(signed_tx.v))
print(w3.toHex(signed_tx.r))
print(w3.toHex(signed_tx.s))

## 4.2 Sign transaction dict via OpenSSL

Prepare to serialize transaction dict (create abstract web3py transaction object), then RLP encode transaction and get keccak hash. This hash should be used in openssl sign.
```bash
echo <keccak_hash> | xxd -r -p > keccak_hash_file
```

In [5]:
unsigned_tx = serializable_unsigned_transaction_from_dict(transaction_dict)
unsigned_tx_keccak_hash = unsigned_tx.hash()
Web3.toHex(unsigned_tx_keccak_hash)

'0x08c863b465cae138e8d532d26075aeb9b615a4a7a5dffa95046cad7382f193d7'

In [6]:
subprocess.run(f'echo {Web3.toHex(unsigned_tx_keccak_hash)[2:]} | xxd -r -p > unsigned_tx_keccak_hash', shell=True).returncode

0

* **s** must be less than ```secp256k1n** / 2 + 1```
* raw recovery ID (**v**) (0/1) role description: value 0 represents an even **y** value of elliptic curve signature, 1 is odd

> it's not possible to get y from openssl, but this value can be only in [0,1]

In [7]:
secp256k1n = 115792089237316195423570985008687907852837564279074904382605163141518161494337
s_num = secp256k1n
while s_num > secp256k1n / 2:
    subprocess.run('openssl pkeyutl -sign -inkey private_key.pem -in unsigned_tx_keccak_hash > signature', shell=True)
    raw_output = subprocess.run('openssl asn1parse -inform DER -in signature', shell=True, capture_output=True).stdout.decode()
    r_str, s_str = [x.split(':')[-1] for x in raw_output.split('\n')[1:3]]
    s_num = int(s_str, 16)
print(r_str)
print(s_str)

E0A8E3763816BC330805FB21C28EB7F6F4321049CD075B6C0188BB08693EDDED
5A1BBF40911D7CD154CB6BF4184820C718F2B3D9221018184ACD92461EC9BE81


Check if signature is valid:

In [8]:
ecdsa_raw_verify(
    Web3.toBytes(unsigned_tx_keccak_hash),
    [int(r_str, 16), int(s_str, 16)],
    private_key_to_public_key(Web3.toBytes(hexstr=private_key_hexstr))
)

True

It must return your ethereum address:
> if address does not match, try to change raw recovery ID (0,1)

In [10]:
v_raw = 0

PublicKey(ecdsa_raw_recover(
    Web3.toBytes(unsigned_tx_keccak_hash),
    [v_raw, int(r_str, 16), int(s_str, 16)],
)).to_address()

'0x90c08087af274b77516df05952273008fea2c4b9'

RLP encode signed transaction, it's final step before you can send it

In [11]:
CHAIN_ID_OFFSET = 35
v = v_raw + CHAIN_ID_OFFSET + 2 * CHAIN_ID # it's from yellow paper

signed_tx = encode_transaction(
    unsigned_tx,
    vrs=(v,
         int(r_str, 16),
         int(s_str, 16)
    )
)
Web3.toHex(signed_tx)

'0xf860048083030d4094bab021f651b216ae46f3f73f75df3c5cb91b631b018025a0e0a8e3763816bc330805fb21c28eb7f6f4321049cd075b6c0188bb08693eddeda05a1bbf40911d7cd154cb6bf4184820c718f2b3d9221018184acd92461ec9be81'

## 5 Send signed transaction 

In [12]:
tx_hash = w3.eth.sendRawTransaction(signed_tx)
tx_hash

HexBytes('0xbae66447a67ffdf1233edb7f6de0546f7ed95baf26acf3328937494b534b1ea1')

In [13]:
w3.eth.getTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0xbae66447a67ffdf1233edb7f6de0546f7ed95baf26acf3328937494b534b1ea1'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x7426f667dae1be28317ab24825878ae5b4898f578304826a458d4bd0892e5401'),
 'blockNumber': 5,
 'from': '0x90c08087af274b77516df05952273008fea2c4b9',
 'to': '0xbab021f651b216ae46f3f73f75df3c5cb91b631b',
 'gasUsed': 21000,
 'cumulativeGasUsed': 21000,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')